# Requirements

In [18]:
import math
import operator

# Problem setting

Structural pattern matching is a very powerful feature that has the potential to make intricate code much easier to write, read and maintain.

Consider the example of an evaluator for arithmetic expressions in reverse Polish notation.  It evalues expressions such as `('+', ('*', 3, 7), 3)` (equivalent to `3*7 + 5`), and raises an exception if an expression can't be matched.

In [80]:
class RpnEvaluator:
    
    def __init__(self):
        self._ops = {
            '+': operator.add,
            '-': operator.sub,
            '*': operator.mul,
            '/': operator.truediv,
            '^': operator.pow,
        }
        self._funcs = {
            'sqrt': math.sqrt,
        }
        
    def is_operator(self, op):
        return op in self._ops
    
    def is_function(self, func):
        return func in self._funcs
    
    def eval(self, expr):
        match expr:
            case (func, arg) if self.is_function(func):
                return self._funcs[func](self.eval(arg))
            case (op, lhs, rhs) if self.is_operator(op):
                return self._ops[op](self.eval(lhs), self.eval(rhs))
            case value if isinstance(value, int) or isinstance(value, float):
                return float(value)
            case _:
                raise ValueError(f'{expr} can not be matched')

In [81]:
evaluator = RpnEvaluator()

In [75]:
evaluator.eval(('+', ('*', 3, 7), 3))

24.0

In [67]:
evaluator.eval(('*', ('sqrt', 5), ('sqrt', 5)))

5.000000000000001

In [70]:
evaluator.eval(('^', 2, 3))

8.0

# Error handling

When you try to evaluate an expression that contains errors, the matching fails.

In [76]:
try:
    evaluate(('+', 'a', 5))
except ValueError as e:
    print(e)

a can not be matched


In [77]:
try:
    evaluator.eval(('**', 3, 5))
except ValueError as e:
    print(e)

('**', 3, 5) can not be matched


In [82]:
try:
    evaluator.eval(('+', 3))
except ValueError as e:
    print(e)

('+', 3) can not be matched
